In [ ]:
experiment_name = "2024.09.17-07_Mistral-7B-instruct-4bit-qlora-fine-tuning___TEST_VALUTAZIONE-MODELLO_FINE-TUNED-FULL"

In [ ]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
# input_data_path_validation = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_DEV_2023.12.11.xlsx"
# input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_2023.12.11.pickle"
# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Mistral/data/output/" + experiment_name
log_filename = base_path + "/../Mistral/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import pickle

In [ ]:
data = data = pd.read_excel(input_data_path)  # THIS TIME WE USE THE TEST DATASET

# data = data.sample(frac=0.005)

data.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [ ]:
def apply_mistral_format(data):
  text = data.text

  template = f"""<s>[INST] You are an helpful virtual assistant specialized in a multi-class single-label text classification task related to the domain of the Sustainable Development Goals.

Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The input text is:
'''
{text}
''' [/INST]

"""

  return template

In [ ]:
data['formatted_prompt'] = data.apply(apply_mistral_format, axis=1)

In [ ]:
data.iloc[0]['formatted_prompt']

'<s>[INST] You are an helpful virtual assistant specialized in a multi-class single-label text classification task related to the domain of the Sustainable Development Goals.\n\nClassify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:\n\n"SDG-1": "End poverty in all its forms everywhere."\n"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."\n"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."\n"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."\n"SDG-5": "Achieve gender equality and empower all women and girls."\n"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."\n"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."\n"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full 

In [ ]:
import glob

def update_data(data_sel):
  drive.mount(mount_point, force_remount=True)

  results = []
  files = glob.glob(output_data_path + f"/*.pkl")
  for file in files:
    res = pd.read_pickle(file)
    results.append(res)
  print(len(results))

  df = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'output'])
  processed_texts = df.text_id.to_list()
  print(processed_texts)

  before = data_sel.shape[0]
  print("Full dataset size:               ", before)
  data = data_sel[~data_sel.text_id.isin(processed_texts)].copy() # DEEP COPY
  after = data.shape[0]
  print("Amount of files to be processed: ", after)
  before - after

  return data.sample(frac=1.00)

# data_sel = update_data(data_sel)
# data_sel.shape[0]

In [ ]:
!pip install -q datasets --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch #, wandb
# from datasets import load_dataset
# from trl import SFTTrainer

In [ ]:
from peft import PeftModel, PeftConfig

base_model = "mistralai/Mistral-7B-Instruct-v0.3"

# config = PeftConfig.from_pretrained("vincenzodeleo/Mistral-7B-Instruct-v0.3-SDG-classification-finetuned")
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model = PeftModel.from_pretrained(model, "vincenzodeleo/Mistral-7B-Instruct-v0.3-SDG-classification-finetuned-FULL", device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(True, True)

In [ ]:
from transformers import pipeline

In [ ]:
messages = [
    {"role": "user", "content": """You are an helpful virtual assistant specialized in a multi-class single-label text classification task related to the domain of the Sustainable Development Goals.\n\nClassify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:\n\n"SDG-1": "End poverty in all its forms everywhere."\n"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."\n"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."\n"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."\n"SDG-5": "Achieve gender equality and empower all women and girls."\n"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."\n"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."\n"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."\n"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."\n"SDG-10": "Reduce inequality within and among countries."\n"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."\n"SDG-12": "Ensure sustainable consumption and production patterns."\n"SDG-13": "Take urgent action to combat climate change and its impacts."\n"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."\n"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."\n"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."\n"SDG-0": "Other."\n\nChoose ONLY ONE label for each input text.\n\nDO NOT include the input text in your answer.\n\nThe input text is:\n\'\'\'\nReporting and progress reports were made to the Minister for Women and Equalities, and the overall report and recommendations of the WBC were delivered to the government and disseminated across UK ministries. The WBC approach is built on the idea that improving gender diversity will lead to greater creativity, problem solving and flexibility, all much-needed skillsets for the UK to adapt to the changing economic environment. Aiming to both increase resources and change organisational culture, WBC took a long-term view, ensuring that longevity and continuity be applied to all suggested strategies.\n\'\'\'\n\n"""},
]
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
chatbot(messages)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

[{'generated_text': [{'role': 'user',
    'content': 'You are an helpful virtual assistant specialized in a multi-class single-label text classification task related to the domain of the Sustainable Development Goals.\n\nClassify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:\n\n"SDG-1": "End poverty in all its forms everywhere."\n"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."\n"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."\n"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."\n"SDG-5": "Achieve gender equality and empower all women and girls."\n"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."\n"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."\n"SDG-8": "Promote sustained, inclu

In [ ]:
data_sel = data

In [ ]:
results = []

while 1:

  data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

  if data_sel.shape[0]==0:
    break


  for index, row in data_sel.iterrows():

    cnt = index

    text_id = row['text_id']
    text = row["formatted_prompt"]
    sdg_true = row["sdg"]

    file = open(log_filename, 'a+', encoding='utf-8')

    print("\n\n##################################################")
    file.write("\n\n##################################################\n")

    print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"])
    file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'])


    messages = [
        {
            "role": "user",
            "content": text
        }
    ]
    chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
    res = chatbot(messages)

    print(res)

    output = res[0]['generated_text'][1]['content']

    print(output)


    result_extended = (cnt, text_id, sdg_true, output)

    print(result_extended)
    file.write(str(result_extended)+'\n')

    with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    file.close()

    break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME

Mounted at /content/gdrive


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

1004
[2755, 1455, 3608, 1968, 2180, 5385, 204, 2301, 1568, 4120, 4639, 1851, 4254, 3098, 391, 3264, 836, 6718, 5693, 4485, 6259, 218, 3732, 4552, 1559, 20, 6209, 1303, 4219, 6684, 1786, 554, 3317, 2293, 3446, 78, 2791, 6479, 1346, 2463, 3571, 2236, 2238, 5813, 646, 2420, 872, 5047, 3019, 433, 3851, 5312, 627, 2575, 98, 3444, 4396, 318, 1661, 1131, 4972, 2889, 4695, 6305, 1516, 4938, 1306, 2094, 5011, 2883, 1049, 3912, 6541, 5493, 6417, 6734, 1446, 2525, 616, 4015, 695, 4375, 5239, 6049, 569, 1005, 92, 4992, 3138, 6667, 1802, 4876, 4116, 706, 280, 6668, 2354, 4572, 768, 5580, 835, 2416, 5094, 6573, 1381, 1565, 5570, 2906, 6242, 1373, 1626, 6471, 5924, 6344, 1535, 5376, 1119, 1794, 3110, 4235, 1912, 6097, 3745, 5790, 4363, 640, 4308, 2747, 5970, 4001, 2400, 913, 471, 4998, 658, 630, 1519, 1388, 5064, 785, 4104, 6628, 3377, 4104, 4075, 665, 5966, 5346, 869, 791, 6100, 3273, 2893, 736, 2949, 2541, 108, 5030, 5667, 3397, 486, 2639, 3614, 2589, 5653, 1866, 1524, 4362, 4057, 3918, 3070, 1947,

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

1005
[2755, 1455, 3608, 1968, 2180, 5385, 204, 2301, 1568, 4120, 4639, 1851, 4254, 3098, 391, 3264, 836, 6718, 5693, 4485, 6259, 218, 3732, 4552, 1559, 20, 6209, 1303, 4219, 6684, 1786, 554, 3317, 2293, 3446, 78, 2791, 6479, 1346, 2463, 3571, 2236, 2238, 5813, 646, 2420, 872, 5047, 3019, 433, 3851, 5312, 627, 2575, 98, 3444, 4396, 318, 1661, 1131, 4972, 2889, 4695, 6305, 1516, 4938, 1306, 2094, 5011, 2883, 1049, 3912, 6541, 5493, 6417, 6734, 1446, 2525, 616, 4015, 695, 4375, 5239, 6049, 569, 1005, 92, 4992, 3138, 6667, 1802, 4876, 4116, 706, 280, 6668, 2354, 4572, 768, 5580, 835, 2416, 5094, 6573, 1381, 1565, 5570, 2906, 6242, 1373, 1626, 6471, 5924, 6344, 1535, 5376, 1119, 1794, 3110, 4235, 1912, 6097, 3745, 5790, 4363, 640, 4308, 2747, 5970, 4001, 2400, 913, 471, 4998, 658, 630, 1519, 1388, 5064, 785, 4104, 6628, 3377, 4104, 4075, 665, 5966, 5346, 869, 791, 6100, 3273, 2893, 736, 2949, 2541, 108, 5030, 5667, 3397, 486, 2639, 3614, 2589, 5653, 1866, 1524, 4362, 4057, 3918, 3070, 1947,

...

1031
[2755, 1455, 3608, 1968, 2180, 5385, 204, 2301, 1568, 4120, 4639, 1851, 4254, 3098, 391, 3264, 836, 6718, 5693, 4485, 6259, 218, 3732, 4552, 1559, 20, 6209, 1303, 4219, 6684, 1786, 554, 3317, 2293, 3446, 78, 2791, 6479, 1346, 2463, 3571, 2236, 2238, 5813, 646, 2420, 872, 5047, 3019, 433, 3851, 5312, 627, 2575, 98, 3444, 4396, 318, 1661, 1131, 4972, 2889, 4695, 6305, 1516, 4938, 1306, 2094, 5011, 2883, 1049, 3912, 6541, 5493, 6417, 6734, 1446, 2525, 616, 4015, 695, 4375, 5239, 6049, 569, 1005, 92, 4992, 3138, 6667, 1802, 4876, 4116, 706, 280, 6668, 2354, 4572, 768, 5580, 835, 2416, 5094, 6573, 1381, 1565, 5570, 2906, 6242, 1373, 1626, 6471, 5924, 6344, 1535, 5376, 1119, 1794, 3110, 4235, 1912, 6097, 3745, 5790, 4363, 640, 4308, 2747, 5970, 4001, 2400, 913, 471, 4998, 658, 630, 1519, 1388, 5064, 785, 4104, 6628, 3377, 4104, 4075, 665, 5966, 5346, 869, 791, 6100, 3273, 2893, 736, 2949, 2541, 108, 5030, 5667, 3397, 486, 2639, 3614, 2589, 5653, 1866, 1524, 4362, 4057, 3918, 3070, 1947,

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

1032
[2755, 1455, 3608, 1968, 2180, 5385, 204, 2301, 1568, 4120, 4639, 1851, 4254, 3098, 391, 3264, 836, 6718, 5693, 4485, 6259, 218, 3732, 4552, 1559, 20, 6209, 1303, 4219, 6684, 1786, 554, 3317, 2293, 3446, 78, 2791, 6479, 1346, 2463, 3571, 2236, 2238, 5813, 646, 2420, 872, 5047, 3019, 433, 3851, 5312, 627, 2575, 98, 3444, 4396, 318, 1661, 1131, 4972, 2889, 4695, 6305, 1516, 4938, 1306, 2094, 5011, 2883, 1049, 3912, 6541, 5493, 6417, 6734, 1446, 2525, 616, 4015, 695, 4375, 5239, 6049, 569, 1005, 92, 4992, 3138, 6667, 1802, 4876, 4116, 706, 280, 6668, 2354, 4572, 768, 5580, 835, 2416, 5094, 6573, 1381, 1565, 5570, 2906, 6242, 1373, 1626, 6471, 5924, 6344, 1535, 5376, 1119, 1794, 3110, 4235, 1912, 6097, 3745, 5790, 4363, 640, 4308, 2747, 5970, 4001, 2400, 913, 471, 4998, 658, 630, 1519, 1388, 5064, 785, 4104, 6628, 3377, 4104, 4075, 665, 5966, 5346, 869, 791, 6100, 3273, 2893, 736, 2949, 2541, 108, 5030, 5667, 3397, 486, 2639, 3614, 2589, 5653, 1866, 1524, 4362, 4057, 3918, 3070, 1947,

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

1034
[2755, 1455, 3608, 1968, 2180, 5385, 204, 2301, 1568, 4120, 4639, 1851, 4254, 3098, 391, 3264, 836, 6718, 5693, 4485, 6259, 218, 3732, 4552, 1559, 20, 6209, 1303, 4219, 6684, 1786, 554, 3317, 2293, 3446, 78, 2791, 6479, 1346, 2463, 3571, 2236, 2238, 5813, 646, 2420, 872, 5047, 3019, 433, 3851, 5312, 627, 2575, 98, 3444, 4396, 318, 1661, 1131, 4972, 2889, 4695, 6305, 1516, 4938, 1306, 2094, 5011, 2883, 1049, 3912, 6541, 5493, 6417, 6734, 1446, 2525, 616, 4015, 695, 4375, 5239, 6049, 569, 1005, 92, 4992, 3138, 6667, 1802, 4876, 4116, 706, 280, 6668, 2354, 4572, 768, 5580, 835, 2416, 5094, 6573, 1381, 1565, 5570, 2906, 6242, 1373, 1626, 6471, 5924, 6344, 1535, 5376, 1119, 1794, 3110, 4235, 1912, 6097, 3745, 5790, 4363, 640, 4308, 2747, 5970, 4001, 2400, 913, 471, 4998, 658, 630, 1519, 1388, 5064, 785, 4104, 6628, 3377, 4104, 4075, 665, 5966, 5346, 869, 791, 6100, 3273, 2893, 736, 2949, 2541, 108, 5030, 5667, 3397, 486, 2639, 3614, 2589, 5653, 1866, 1524, 4362, 4057, 3918, 3070, 1947,